Building clusterisation algorithm for a Uralsib's sales department

**Step 1**. Parse data from 1 community for 1 user: <br>
1. Getting commentator's id
2. Getting commentator's `name`, `surname`, `id`, [`city`], [`age`], `BankName`, `comment`, [`likes`] <br>

**Step 2**. Parse data from 1 community for multiple users: <br>
1. Building dataframe of commentators <br>

**Step 3**. Parse data from multiple communities: <br>
1. Building multiple dataframes (one df for each bank)
2. Concatenating the dataframes <br>

**Step 4**. Add `sentiment` to all users <br>
1. Analyse the database and prepare clusters & rules 
2. Prepare 'sentiment piggy bank' for different sentiments
3. Label commentators based on their comments (it might be 'undefined' group - for further operator's investigation)

**Step 5**. Prepare presentation 
1. Prepare slides 

In [2]:
import requests 
import time
import json
import pandas as pd
import numpy as np
import re

In [4]:
pd.options.display.max_colwidth = 300

#### Getting post ids

In [2]:
token = '#' ## put your token
version = 5.92
domain = 'sber'
count_posts = 100
count_comments = 100

In [4]:
response = requests.get('https://api.vk.com/method/wall.get',
                        params = {
                            'access_token' : token,
                            'v' : version,
                            'domain' : domain,
                            'count': count_posts
                        }
                       )

In [5]:
posts = response.json()["response"]["items"]

In [6]:
#print(posts)

In [7]:
## to read in human language
posts_readable = json.dumps(posts, indent = 2, ensure_ascii=False)
print(posts_readable)

[
  {
    "id": 2096781,
    "from_id": -22522055,
    "owner_id": -22522055,
    "date": 1654866002,
    "marked_as_ads": 0,
    "post_type": "post",
    "text": "Выбираете в СберМаркете садовые качели, новый мангал, грунт или семена бархатцев — оплатите покупки картой СберБанка и получите до 10% от чека бонусами.\n\nПредложение действует до 5 июля и доступно всем, кто уже участвует в программе «СберСпасибо».\n\nУзнайте подробнее об акции и зарегистрируйтесь в программе, если не сделали этого раньше: http://sber.me/?p=vPL61\n\n#экосистема@sber",
    "attachments": [
      {
        "type": "photo",
        "photo": {
          "album_id": -7,
          "date": 1654864246,
          "id": 457303986,
          "owner_id": -22522055,
          "access_key": "28e857fb3369b7011c",
          "post_id": 2096781,
          "sizes": [
            {
              "height": 55,
              "url": "https://sun9-30.userapi.com/s/v1/ig2/9ZwcCJRLOg9Pdf3MJWve7gsS0TErCQJYY1QCTBLtkFNPRr3D3bLlvWW2k2y3

In [8]:
posts[0]['id']

2096781

In [9]:
posts_id = []

counter = 0
while counter <= count_posts:
    for item in posts:
        try:
            posts_id.append(item['id'])
            counter += 1
            #print(counter)
            
        except:
            pass

In [10]:
post_ids_set = set(posts_id)    
#print(post_ids)
print(len(post_ids_set))

100


#### Getting commentators ids

In [11]:
all_comments = []

In [12]:
for post_id in post_ids_set:
    try:
        response = requests.get('https://api.vk.com/method/wall.getComments',
                                params = {
                                    'access_token' : token,
                                    'owner_id': -22522055,
                                    'v' : version,
                                    'domain' : domain,
                                    'post_id': post_id,
                                    'count': count_comments
                                }
                               )

        comments = response.json()["response"]["items"]
        all_comments.append(comments)
        time.sleep(0.5)
    except:
        pass

In [13]:
print(all_comments)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
user_id = all_comments[2][0]['from_id']
print("USER: ", user_id)

text = all_comments[2][0]['text']
print("TEXT: ", text)

USER:  614027635
TEXT:  Первый


In [15]:
user_ids = []
user_texts = []

for comment_list in all_comments:
    for item in comment_list:
        
        
        user_ids.append(item['from_id'])
        user_texts.append(item['text'])
        #print(item['from_id'])

print(len(user_texts))
#print(user_texts)        
print(len(user_ids))
#print(user_ids)

5840
5840


In [16]:
users_df = pd.DataFrame(
    {'user_id': user_ids,
     'comment': user_texts,
    })

In [17]:
users_df.head()

,user_id,comment
0,712644268,"Лучше всего хранить деньги в кошельке, чем в б..."
1,22074792,С вашими то процентами и сохранить?!? Вы смеёт...
2,9550188,Были бы деньги 😵
3,710618222,а с какой целью опрос что дома сколько и где л...
4,493222577,Публичный опрос специально для того чтобы друз...


In [ ]:
users_df.to_excel('users.xlsx', index = False)

#### Уралсиб

In [59]:
df = pd.read_csv("uralsib_neg.csv", delimiter=';', quoting=3, error_bad_lines=False)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [60]:
df.head(5)

,Ссылка,Текст,Тип,Автор,"""Ссылка на автора""",Дата,Лайков,Комментариев,Репостов,Переходов,"""Просмотров вики страницы""",Просмотров,"""Лайков / Просмотров""","""Комментариев / Просмотров""","""Репостов / Просмотров""","""Переходов / Просмотров""","""Просмотров вики страницы / Просмотров поста"""
0,https://vk.com/wall-35524999_24234?reply=24234,"""[id729474732|Михаил], абсолютно верно. Звонила по 8-800 - ничего сказать не могут. Написала претензию - зарегистрировали только после того, как написала повторно, написала претензию на сайте банка и здесь. Прислали ответ - обычная отписка. Ни один документ, предоставленный мною не рассмотрен. П...",Комментарий,"""Нина Пантелеймон""",https://vk.com/id346310153,"""11:37 01.06.2022""",0.0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
1,https://vk.com/wall-35524999_24218?reply=24218,"""Дорогие недрузья, спасибо за потраченный день зря, одобрили кредит на машину, заплатили все уже, документы подписали, через пол часа приходит отказ. Мы потратили с мужем весь день, вы видели всю информацию и кредит одобрили. Потом отказали на основании той же информации. Вы себя считаете профес...",Комментарий,"""Настя Ивойлова""",https://vk.com/id3215644,"""18:46 29.05.2022""",0.0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
2,https://vk.com/wall-35524999_24203?reply=24203,"""Сегодня опять не работает !!!!!задолбали!!!!!""",Комментарий,"""Евгения Овсиенко""",https://vk.com/id28979461,"""14:14 27.05.2022""",0.0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
3,https://vk.com/wall-35524999_24202?reply=24202,"""Ужасный банк с постоянными сбоями причём сутками не работает !!!! Вчера не смогли перевести деньги поставщику и попали на крупную сумму ((( ответственности у них нет !!!! Не рекомендую этот банк и сами уходить от них будем!!!!""",Комментарий,"""Евгения Овсиенко""",https://vk.com/id28979461,"""14:13 27.05.2022""",4.0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
4,https://vk.com/wall-35524999_24195?reply=24195,"""Здравствуйте!",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
df.dropna(subset=['"Ссылка на автора"'], inplace=True)

cumulative_cols_of_interest = ['Ссылка', 'Текст', 'Тип', 'Автор', '"Ссылка на автора"',  'Дата', 'Лайков']
df = df[cumulative_cols_of_interest]

df.rename(columns={'Ссылка': 'Comment_Link', 'Текст': 'Text', 'Тип': 'Type', 'Автор': 'Author', '"Ссылка на автора"': 'Author_Link', 'Дата': 'Date', 'Лайков': 'Likes'}, inplace=True)

In [53]:
df = df[df['Type'] == 'Комментарий']
df = df[df['Type'] != '"Банк Уралсиб"']

df['Bank'] = 'Уралсиб'
df['Sentiment'] = 'negative_Uralsib'

In [54]:
print(df.shape)
df = df.reset_index(drop = True)
df

(41, 9)


,Comment_Link,Text,Type,Author,Author_Link,Date,Likes,Bank,Sentiment
0,https://vk.com/wall-35524999_24234?reply=24234,"""[id729474732|Михаил], абсолютно верно. Звонил...",Комментарий,"""Нина Пантелеймон""",https://vk.com/id346310153,"""11:37 01.06.2022""",0.0,Уралсиб,negative_Uralsib
1,https://vk.com/wall-35524999_24218?reply=24218,"""Дорогие недрузья, спасибо за потраченный день...",Комментарий,"""Настя Ивойлова""",https://vk.com/id3215644,"""18:46 29.05.2022""",0.0,Уралсиб,negative_Uralsib
2,https://vk.com/wall-35524999_24203?reply=24203,"""Сегодня опять не работает !!!!!задолбали!!!!!""",Комментарий,"""Евгения Овсиенко""",https://vk.com/id28979461,"""14:14 27.05.2022""",0.0,Уралсиб,negative_Uralsib
3,https://vk.com/wall-35524999_24202?reply=24202,"""Ужасный банк с постоянными сбоями причём сутк...",Комментарий,"""Евгения Овсиенко""",https://vk.com/id28979461,"""14:13 27.05.2022""",4.0,Уралсиб,negative_Uralsib
4,https://vk.com/wall-35524999_24194?reply=24194,"""[club35524999|Банк Уралсиб], подскажите когда...",Комментарий,"""Надежда Воробьёва""",https://vk.com/id2810021,"""10:53 27.05.2022""",1.0,Уралсиб,negative_Uralsib
5,https://vk.com/wall-35524999_24192?reply=24192,"""Сегодня Уралсиб бизнес онлайн опять не работает""",Комментарий,"""Анастасия Поварницина""",https://vk.com/id74630653,"""10:29 27.05.2022""",0.0,Уралсиб,negative_Uralsib
6,https://vk.com/wall-35524999_24180?reply=24180,"""Весь день не могу попасть в личный кабинет. С...",Комментарий,"""Анастасия Поварницина""",https://vk.com/id74630653,"""13:57 26.05.2022""",0.0,Уралсиб,negative_Uralsib
7,https://vk.com/wall-35524999_24219?reply=24219,"""[id666640950|Lena], замечательно, недееспособ...",Комментарий,"""Настя Ивойлова""",https://vk.com/id3215644,"""19:11 29.05.2022""",0.0,Уралсиб,negative_Uralsib
8,https://vk.com/wall-35524999_24137?reply=24137,"""Как закончить издевательство со стороны банка...",Комментарий,"""Lena Miller""",https://vk.com/id666640950,"""12:46 19.05.2022""",0.0,Уралсиб,negative_Uralsib
9,https://vk.com/wall-35524999_24106?reply=24106,"""Добрый день. Читаю ваши тарифы Обслуживание д...",Комментарий,"""Валентина Шарошкина""",https://vk.com/id78376477,"""13:24 18.05.2022""",1.0,Уралсиб,negative_Uralsib


In [49]:
df.to_excel('uralsib_neg_fin.xlsx', index = False)

#### Конкурент (Сбербанк)

In [8]:
df2 = pd.read_csv("sber_neg.csv", delimiter=';', quoting=3, error_bad_lines=False)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 3053: expected 17 fields, saw 18\nSkipping line 3233: expected 17 fields, saw 18\nSkipping line 3487: expected 17 fields, saw 18\nSkipping line 3637: expected 17 fields, saw 18\nSkipping line 3665: expected 17 fields, saw 18\nSkipping line 7430: expected 17 fields, saw 18\nSkipping line 7847: expected 17 fields, saw 18\n'


In [9]:
df2.head(5)

,Ссылка,Текст,Тип,Автор,"""Ссылка на автора""",Дата,Лайков,Комментариев,Репостов,Переходов,"""Просмотров вики страницы""",Просмотров,"""Лайков / Просмотров""","""Комментариев / Просмотров""","""Репостов / Просмотров""","""Переходов / Просмотров""","""Просмотров вики страницы / Просмотров поста"""
0,https://vk.com/wall-22522055_2096920?reply=209...,"""[id423827102|Иван], а это сбой! днем работало...",Комментарий,"""Евгений Александрович""",https://vk.com/id396323737,"""22:20 10.06.2022""",0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
1,https://vk.com/wall-22522055_2096917?reply=209...,"""[id396323737|Евгений], я не сказал что у меня...",Комментарий,"""Иван Иванов""",https://vk.com/id423827102,"""22:11 10.06.2022""",0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
2,https://vk.com/wall-22522055_2096916?reply=209...,"""[id423827102|Иван], нет но так сказали у меня...",Комментарий,"""Евгений Александрович""",https://vk.com/id396323737,"""22:08 10.06.2022""",0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
3,https://vk.com/wall-22522055_2096912?reply=209...,"""[id423827102|Иван], взял айфон и хвалишся? бл...",Комментарий,"""Евгений Александрович""",https://vk.com/id396323737,"""21:54 10.06.2022""",0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
4,https://vk.com/wall-22522055_2096880?reply=209...,"""Это норма? Уже 5 часов не могу""",Комментарий,"""Никита Чернышов""",https://vk.com/id717656379,"""19:51 10.06.2022""",0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%


In [10]:
df2.dropna(subset=['"Ссылка на автора"'], inplace=True)

cumulative_cols_of_interest = ['Ссылка', 'Текст', 'Тип', 'Автор', '"Ссылка на автора"',  'Дата', 'Лайков']
df2 = df2[cumulative_cols_of_interest]

df2.rename(columns={'Ссылка': 'Comment_Link', 'Текст': 'Text', 'Тип': 'Type', 'Автор': 'Author', '"Ссылка на автора"': 'Author_Link', 'Дата': 'Date', 'Лайков': 'Likes'}, inplace=True)

In [11]:
df2 = df2[df2['Type'] == 'Комментарий']
df2

,Comment_Link,Text,Type,Author,Author_Link,Date,Likes
0,https://vk.com/wall-22522055_2096920?reply=209...,"""[id423827102|Иван], а это сбой! днем работало...",Комментарий,"""Евгений Александрович""",https://vk.com/id396323737,"""22:20 10.06.2022""",0
1,https://vk.com/wall-22522055_2096917?reply=209...,"""[id396323737|Евгений], я не сказал что у меня...",Комментарий,"""Иван Иванов""",https://vk.com/id423827102,"""22:11 10.06.2022""",0
2,https://vk.com/wall-22522055_2096916?reply=209...,"""[id423827102|Иван], нет но так сказали у меня...",Комментарий,"""Евгений Александрович""",https://vk.com/id396323737,"""22:08 10.06.2022""",0
3,https://vk.com/wall-22522055_2096912?reply=209...,"""[id423827102|Иван], взял айфон и хвалишся? бл...",Комментарий,"""Евгений Александрович""",https://vk.com/id396323737,"""21:54 10.06.2022""",0
4,https://vk.com/wall-22522055_2096880?reply=209...,"""Это норма? Уже 5 часов не могу""",Комментарий,"""Никита Чернышов""",https://vk.com/id717656379,"""19:51 10.06.2022""",0
...,...,...,...,...,...,...,...
7821,https://vk.com/wall-22522055_2054918?reply=205...,"""[id271853141|Дмитрий], если Сбербанк и обанкр...",Комментарий,"""Василиса Кузнецова""",https://vk.com/id631498689,"""17:59 29.01.2022""",0
7822,https://vk.com/wall-22522055_2054916?reply=205...,"""[id631498689|Василиса], надёжность банка поня...",Комментарий,"""Дмитрий Марков""",https://vk.com/id271853141,"""17:55 29.01.2022""",0
7823,https://vk.com/wall-22522055_2091797?reply=209...,"""[id429984592|Евгений], пришлите, пожалуйста, ...",Комментарий,Сбер,https://vk.com/club22522055,"""14:06 11.05.2022""",0
7824,https://vk.com/wall-22522055_2091770?reply=209...,"""Здравствуйте. приложение не видит камеру и ми...",Комментарий,"""Евгений Евсеев""",https://vk.com/id429984592,"""11:21 11.05.2022""",0


In [12]:
df2 = df2[df2['Type'] != 'Сбер']

df2['Bank'] = 'Сбербанк'
df2['Sentiment'] = 'negative_Sberbank'

In [13]:
print(df2.shape)
df2

(5366, 9)


,Comment_Link,Text,Type,Author,Author_Link,Date,Likes,Bank,Sentiment
0,https://vk.com/wall-22522055_2096920?reply=209...,"""[id423827102|Иван], а это сбой! днем работало...",Комментарий,"""Евгений Александрович""",https://vk.com/id396323737,"""22:20 10.06.2022""",0,Сбербанк,negative_Sberbank
1,https://vk.com/wall-22522055_2096917?reply=209...,"""[id396323737|Евгений], я не сказал что у меня...",Комментарий,"""Иван Иванов""",https://vk.com/id423827102,"""22:11 10.06.2022""",0,Сбербанк,negative_Sberbank
2,https://vk.com/wall-22522055_2096916?reply=209...,"""[id423827102|Иван], нет но так сказали у меня...",Комментарий,"""Евгений Александрович""",https://vk.com/id396323737,"""22:08 10.06.2022""",0,Сбербанк,negative_Sberbank
3,https://vk.com/wall-22522055_2096912?reply=209...,"""[id423827102|Иван], взял айфон и хвалишся? бл...",Комментарий,"""Евгений Александрович""",https://vk.com/id396323737,"""21:54 10.06.2022""",0,Сбербанк,negative_Sberbank
4,https://vk.com/wall-22522055_2096880?reply=209...,"""Это норма? Уже 5 часов не могу""",Комментарий,"""Никита Чернышов""",https://vk.com/id717656379,"""19:51 10.06.2022""",0,Сбербанк,negative_Sberbank
...,...,...,...,...,...,...,...,...,...
7821,https://vk.com/wall-22522055_2054918?reply=205...,"""[id271853141|Дмитрий], если Сбербанк и обанкр...",Комментарий,"""Василиса Кузнецова""",https://vk.com/id631498689,"""17:59 29.01.2022""",0,Сбербанк,negative_Sberbank
7822,https://vk.com/wall-22522055_2054916?reply=205...,"""[id631498689|Василиса], надёжность банка поня...",Комментарий,"""Дмитрий Марков""",https://vk.com/id271853141,"""17:55 29.01.2022""",0,Сбербанк,negative_Sberbank
7823,https://vk.com/wall-22522055_2091797?reply=209...,"""[id429984592|Евгений], пришлите, пожалуйста, ...",Комментарий,Сбер,https://vk.com/club22522055,"""14:06 11.05.2022""",0,Сбербанк,negative_Sberbank
7824,https://vk.com/wall-22522055_2091770?reply=209...,"""Здравствуйте. приложение не видит камеру и ми...",Комментарий,"""Евгений Евсеев""",https://vk.com/id429984592,"""11:21 11.05.2022""",0,Сбербанк,negative_Sberbank


In [14]:
df2.to_excel('sberbank_neg_fin.xlsx', index = False)

#### Банк Уралсиб - позитивные кейсы

In [31]:
df3 = pd.read_csv("uralsib_positive.csv", delimiter=';', quoting=3, error_bad_lines=False)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [32]:
df3

,Ссылка,Текст,Тип,Автор,"""Ссылка на автора""",Дата,Лайков,Комментариев,Репостов,Переходов,"""Просмотров вики страницы""",Просмотров,"""Лайков / Просмотров""","""Комментариев / Просмотров""","""Репостов / Просмотров""","""Переходов / Просмотров""","""Просмотров вики страницы / Просмотров поста"""
0,https://vk.com/wall-35524999_24218?reply=24218,"""Дорогие недрузья, спасибо за потраченный день...",Комментарий,"""Настя Ивойлова""",https://vk.com/id3215644,"""18:46 29.05.2022""",0.0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
1,https://vk.com/wall-35524999_24211?reply=24211,"""Не рекомендую ваш банк !!!!!!!!""",Комментарий,"""Евгения Овсиенко""",https://vk.com/id28979461,"""15:38 27.05.2022""",1.0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
2,https://vk.com/wall-35524999_24202?reply=24202,"""Ужасный банк с постоянными сбоями причём сутк...",Комментарий,"""Евгения Овсиенко""",https://vk.com/id28979461,"""14:13 27.05.2022""",4.0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
3,https://vk.com/wall-35524999_24156?reply=24156,"""[id102165009|Алексей], спасибо, если банк не ...",Комментарий,"""Lena Miller""",https://vk.com/id666640950,"""21:27 19.05.2022""",0.0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
4,https://vk.com/wall-35524999_23602?reply=23602,"""[id3055571|Мария], спасибо, что уведомили! Хо...",Комментарий,"""Банк Уралсиб""",https://vk.com/club35524999,"""16:06 10.03.2022""",0.0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
5,https://vk.com/wall-35524999_23599?reply=23599,"""[club35524999|Банк Уралсиб], уже разобралась ...",Комментарий,"""Мария Наприенко""",https://vk.com/id3055571,"""15:57 10.03.2022""",0.0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
6,https://vk.com/wall-35524999_23516?reply=23516,"""Здравствуйте, Оплатил авиабилет частично бону...",Комментарий,"""Дмитрий Недвига""",https://vk.com/id177713971,"""19:09 04.03.2022""",0.0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
7,https://vk.com/wall-35524999_23515?reply=23515,"""[club35524999|Банк Уралсиб], спасибо, но тепе...",Комментарий,"""Елена Балачина""",https://vk.com/id18480814,"""18:49 04.03.2022""",0.0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
8,https://vk.com/wall-35524999_23503?reply=23503,"""[id18480814|Елена], спасибо, что сообщили. Вм...",Комментарий,"""Банк Уралсиб""",https://vk.com/club35524999,"""18:19 04.03.2022""",0.0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
9,https://vk.com/wall-35524999_24253?reply=24253,"""я живу в Приднестровье и наш банк выдал мне к...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df3.to_excel('uralsib_pos.xlsx', index = False)

manual corrections

In [34]:
df3 = pd.read_excel("uralsib_pos.xlsx")

In [35]:
df3

,Ссылка,Текст,Тип,Автор,"""Ссылка на автора""",Дата,Лайков,Комментариев,Репостов,Переходов,"""Просмотров вики страницы""",Просмотров,"""Лайков / Просмотров""","""Комментариев / Просмотров""","""Репостов / Просмотров""","""Переходов / Просмотров""","""Просмотров вики страницы / Просмотров поста"""
0,https://vk.com/wall-35524999_23602?reply=23602,"""[id3055571|Мария], спасибо, что уведомили! Хо...",Комментарий,"""Банк Уралсиб""",https://vk.com/club35524999,"""16:06 10.03.2022""",0.0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
1,https://vk.com/wall-35524999_23599?reply=23599,"""[club35524999|Банк Уралсиб], уже разобралась ...",Комментарий,"""Мария Наприенко""",https://vk.com/id3055571,"""15:57 10.03.2022""",0.0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
2,https://vk.com/wall-35524999_23515?reply=23515,"""[club35524999|Банк Уралсиб], спасибо, но тепе...",Комментарий,"""Елена Балачина""",https://vk.com/id18480814,"""18:49 04.03.2022""",0.0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
3,https://vk.com/wall-35524999_23503?reply=23503,"""[id18480814|Елена], спасибо, что сообщили. Вм...",Комментарий,"""Банк Уралсиб""",https://vk.com/club35524999,"""18:19 04.03.2022""",0.0,0.0,0.0,0.0,0.0,0,0%,0%,0%,0%,0%
4,https://vk.com/wall-35524999_24253?reply=24253,"""я живу в Приднестровье и наш банк выдал мне к...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,или может я могу получить просто swift данные ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"спасибо""",Комментарий,"""David Dark""",https://vk.com/id77789476,"""02:40 05.06.2022""",0,0.0,0.0,0.0,0.0,0.0,0%,0%,0%,0%,0%,NaN
7,https://vk.com/wall-35524999_24255?reply=24255,"""я живу в Приднестровье и наш банк выдал мне к...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,или может я могу получить просто swift данные ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"спасибо""",Комментарий,"""David Dark""",https://vk.com/id77789476,"""02:40 05.06.2022""",1,0.0,0.0,0.0,0.0,0.0,0%,0%,0%,0%,0%,NaN


## Кластеризация

Clusters: <br>
1. Вклады
2. Кредитные карты
3. Кредиты
4. Дебитовые карты
5. Веб-сайт и приложение

In [41]:
import pandas as pd
import os
import numpy as np
import re

In [51]:
pd.options.display.max_colwidth = 300

In [42]:
df = pd.read_excel('uralsib_neg_fin.xlsx')

In [43]:
vklad_strings = ("вклад|счет|вложение|депозит")

credit_card_strings = ("кредитн|кредитка")
credit_strings = ("кредит")

debit_strings = ("дебитовая")

web_and_app_strings = ("приложен|сайт")

In [44]:
vklad_cond = df['Text'].str.contains(vklad_strings)

credit_card_cond = df['Text'].str.contains(credit_card_strings)
credit_cond = df['Text'].str.contains(credit_strings)

debit_cond = df['Text'].str.contains(debit_strings)

web_and_app_cond = df['Text'].str.contains(web_and_app_strings)

In [45]:
credit = (credit_cond & (~credit_card_cond))

In [46]:
df[["Cluster1", "Cluster2", "Cluster3"]] = np.nan

In [47]:
def fill_label(df, cond, val):
    
    for idx in df[cond].index:
        v1 = df.loc[idx, "Cluster1"]
        
        if (v1 == val):
            continue   

        if isinstance(v1, float) and np.isnan(v1):

            df.loc[idx, "Cluster1"] = val
            continue
        
        
        v2 = df.loc[idx, "Cluster2"]
        if (v2 == val):
            continue

        if isinstance(v2, float) and np.isnan(v2):
            df.loc[idx, "Cluster2"] = val
            continue
        

        df.loc[idx, "Cluster3"] = val

In [48]:
fill_label(df, vklad_cond, "Вклад")
fill_label(df, credit_card_cond, "Кредитная карта")
fill_label(df, credit, "Кредит")

fill_label(df, debit_cond, "Дебитная карта")
fill_label(df, web_and_app_cond, "Приложение и Веб-Сайт")

In [49]:
df.dropna(subset=['Cluster1'], inplace=True)

In [50]:
df

,index,Comment_Link,Text,Type,Author,Author_Link,Date,Likes,Bank,Sentiment,Cluster1,Cluster2,Cluster3
0,0,https://vk.com/wall-35524999_24234?reply=24234,"""[id729474732|Михаил], абсолютно верно. Звонил...",Комментарий,"""Нина Пантелеймон""",https://vk.com/id346310153,"""11:37 01.06.2022""",0,Уралсиб,negative_Uralsib,Вклад,Кредитная карта,Приложение и Веб-Сайт
1,1,https://vk.com/wall-35524999_24218?reply=24218,"""Дорогие недрузья, спасибо за потраченный день...",Комментарий,"""Настя Ивойлова""",https://vk.com/id3215644,"""18:46 29.05.2022""",0,Уралсиб,negative_Uralsib,Кредит,NaN,NaN
4,10,https://vk.com/wall-35524999_24194?reply=24194,"""[club35524999|Банк Уралсиб], подскажите когда...",Комментарий,"""Надежда Воробьёва""",https://vk.com/id2810021,"""10:53 27.05.2022""",1,Уралсиб,negative_Uralsib,Приложение и Веб-Сайт,NaN,NaN
7,13,https://vk.com/wall-35524999_24219?reply=24219,"""[id666640950|Lena], замечательно, недееспособ...",Комментарий,"""Настя Ивойлова""",https://vk.com/id3215644,"""19:11 29.05.2022""",0,Уралсиб,negative_Uralsib,Кредит,NaN,NaN
8,14,https://vk.com/wall-35524999_24137?reply=24137,"""Как закончить издевательство со стороны банка...",Комментарий,"""Lena Miller""",https://vk.com/id666640950,"""12:46 19.05.2022""",0,Уралсиб,negative_Uralsib,Кредит,NaN,NaN
9,15,https://vk.com/wall-35524999_24106?reply=24106,"""Добрый день. Читаю ваши тарифы Обслуживание д...",Комментарий,"""Валентина Шарошкина""",https://vk.com/id78376477,"""13:24 18.05.2022""",1,Уралсиб,negative_Uralsib,Вклад,Кредитная карта,Приложение и Веб-Сайт
13,23,https://vk.com/wall-35524999_23499?reply=23499,"""сайт не работает""",Комментарий,"""Оксана Сав""",https://vk.com/id348812781,"""16:39 04.03.2022""",0,Уралсиб,negative_Uralsib,Приложение и Веб-Сайт,NaN,NaN
16,26,https://vk.com/wall-35524999_23495?reply=23495,"""Не работает ни онлайн-банк, ни приложение. Ни...",Комментарий,"""Юлия Мелькина""",https://vk.com/id539135305,"""15:26 04.03.2022""",3,Уралсиб,negative_Uralsib,Вклад,Приложение и Веб-Сайт,NaN
17,31,https://vk.com/wall-35524999_24051?reply=24051,"""Очень жаль, что информация на сайте банка не ...",Комментарий,"""Ирина Асташева""",https://vk.com/id232679907,"""11:51 04.05.2022""",0,Уралсиб,negative_Uralsib,Вклад,Кредитная карта,Приложение и Веб-Сайт
18,32,https://vk.com/wall-35524999_24135?reply=24135,"""Как закончить издевательство со стороны банка...",Комментарий,"""Lena Miller""",https://vk.com/id666640950,"""12:45 19.05.2022""",0,Уралсиб,negative_Uralsib,Кредит,NaN,NaN


## Создание коммьюнити - рекомендации для sales-специалиста

In [13]:
df = pd.read_excel('Users of community.xlsx')

In [14]:
df.head(6)

,Имя,user_id,Банк,Сантимент,Кластер в сообществе Уралсиб,Потребительская активность,Рекомендация для sales-специалиста
0,Павел,025a2595-8b57-46f5-a98c-e6a4ca407ca3,Уралсиб,Лояльный клиент Уралсиб,Предприниматель - Ищу инвестора / кредитора,Импульсивный транжира,Предложить кредит
1,Ольга,030e0849-f317-4958-9924-210f4f85e865,Альфабанк,Недовольный клиент банка-конкурента,Топ-менеджер - Ищу подрядчиков,Осмотрительный планировщик,Предложить зарплатный проект
2,Юлия,041a33f4-6427-42b2-b336-5e1da8a4ae43,Тинькоф,Заинтересован в банке Уралсиб,Предприниматель - Ищу клиента,Домосед-консерватор,Предложить выгодный обменный курс
3,Мария,046eb433-8d4b-4468-bab8-2d13b4c750d7,Уралсиб,Недовольный клиент Уралсиб,Предприниматель - Ищу технических экспертов,Цифровой энтузиаст,Предложить кредитную карту
4,Екатерина,077c4e74-ba66-4e47-be0d-af27a64f9b8e,ВТБ,Недовольный клиент банка-конкурента,Самозанятый - Ищу клиентов,Импульсивный транжира,Предложить автокредит
5,Петр,095e70bb-ea44-40c6-a58c-d77233a934ce,Роскомснаббанк,Заинтересован в банке Уралсиб,Специалист с высоким уровнем заработка - ищу партнеров,Осмотрительный планировщик,Предложить ипотеку
